In [155]:
import numpy as np
from numpy.random import randn
from numpy.random import randint
import os
import cv2
import pandas as pd
import gzip
from matplotlib import pyplot as plt
from keras.preprocessing.image import load_img
from keras.datasets.mnist import load_data
from keras.preprocessing.image import img_to_array
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Concatenate




# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # get noise and label inputs from generator model
    gen_noise, gen_label = g_model.input
    # get image output from the generator model
    gen_output = g_model.output
    # connect image output and label input from generator as inputs to discriminator
    gan_output = d_model([gen_output, gen_label])
    # define gan model as taking noise and label and outputting a classification
    model = Model([gen_noise, gen_label], gan_output)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [220]:
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    images = images.astype('float32')
    images = images.reshape(60000, 28, 28, 1)
    # scale from [0,255] to [-1,1]
    images = (images - 127.5) / 127.5
    return (images, np.vstack((labels, labels)).T)

# select real samples
def generate_real_samples(dataset, n_samples):
    # split into images and labels
    images, labels = dataset
    # choose random instances
    ix = randint(0, images.shape[0], n_samples)
    # select images and labels
    X, labels = images[ix], labels[ix]
    # generate class labels
    y = np.ones((n_samples, 1))
    #print('X.shape: ', X.shape, 'labels.shape: ', labels.shape, 'y.shape: ', y.shape)
    return [X, labels], y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=10):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    z_input = x_input.reshape(n_samples, latent_dim) # (64, 100)
    # generate labels
    labels = randint(0, n_classes, n_samples)
    return [z_input, np.vstack((labels, labels)).T]

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    z_input, labels_input = generate_latent_points(latent_dim, n_samples)
    #print('z_input: ', z_input.shape, 'labels_input: ', labels_input.shape)
    # predict outputs
    images = generator.predict([z_input, labels_input])
    # create class labels
    y = np.zeros((n_samples, 1))
    #print('images: ', images.shape, 'labels_input: ', labels_input.shape, 'y: ', y.shape)
    return [images, labels_input], y

# load fashion mnist images
def load_real_samples():
    # load dataset
    (trainX, trainy), (_, _) = load_data()
    # expand to 3d, e.g. add channels
    X = np.expand_dims(trainX, axis=-1)
    # convert from ints to floats
    X = X.astype('float32')
    # scale from [0,255] to [-1,1]
    X = (X - 127.5) / 127.5
    return [X, trainy]


In [225]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=5, n_batch=128):
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected ✬real✬ samples
            [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
            # update discriminator model weights
            d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real)
            # generate ✬fake✬ examples
            
            [X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # update discriminator model weights
            d_loss2, _ = d_model.train_on_batch([X_fake, labels], y_fake)
            # prepare points in latent space as input for the generator
            [z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = np.ones((n_batch, 1))
            # update the generator via the discriminator✬s error
            g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)
            
            # summarize loss on this batch
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' % (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
    # save the generator model
    g_model.save('cgan_models/cgan_generator.h5')

In [226]:
# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1), n_classes=10):
    # label input
    in_label = Input(shape=(2,))
    # embedding for categorical input
    li = Embedding(n_classes, 50)(in_label)
    # scale up to image dimensions with linear activation
    n_nodes = in_shape[0] * in_shape[1]
    li = Dense(n_nodes)(li)
    # reshape to additional channel
    li = Reshape((in_shape[0], in_shape[1], 2))(li)
    # image input
    in_image = Input(shape=in_shape)
    # concat label as a channel
    merge = Concatenate()([in_image, li])
    # downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(merge)
    fe = LeakyReLU(alpha=0.2)(fe)
    # downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    # flatten feature maps
    fe = Flatten()(fe)
    # dropout
    fe = Dropout(0.4)(fe)
    # output
    out_layer = Dense(1, activation='sigmoid')(fe)
    # define model
    model = Model([in_image, in_label], out_layer)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# define the standalone generator model
def define_generator(latent_dim, n_classes=10):
    # label input
    in_label = Input(shape=(2,))
    # embedding for categorical input
    li = Embedding(n_classes, 50)(in_label)
    # linear multiplication
    n_nodes = 7 * 7
    li = Dense(n_nodes)(li)
    # reshape to additional channel
    li = Reshape((7, 7, 2))(li)
    # image generator input
    in_lat = Input(shape=(latent_dim,))
    # foundation for 7x7 image
    n_nodes = 128 * 7 * 7
    gen = Dense(n_nodes)(in_lat)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Reshape((7, 7, 128))(gen)
    # merge image gen and label input
    merge = Concatenate()([gen, li])
    # upsample to 14x14
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(merge)
    gen = LeakyReLU(alpha=0.2)(gen)
    # upsample to 28x28
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    # output
    out_layer = Conv2D(1, (7,7), activation='tanh', padding='same')(gen)
    # define model
    model = Model([in_lat, in_label], out_layer)
    return model

In [227]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
#dataset = my_custom_load(59992)
dataset = load_mnist('fash', kind='train')

In [228]:
# train model
train(g_model, d_model, gan_model, dataset[:15000], latent_dim);

>1, 1/468, d1=0.706, d2=0.696 g=0.691
>1, 2/468, d1=0.639, d2=0.699 g=0.688
>1, 3/468, d1=0.566, d2=0.704 g=0.684
>1, 4/468, d1=0.518, d2=0.712 g=0.677
>1, 5/468, d1=0.459, d2=0.723 g=0.668
>1, 6/468, d1=0.405, d2=0.741 g=0.652
>1, 7/468, d1=0.352, d2=0.774 g=0.632
>1, 8/468, d1=0.310, d2=0.811 g=0.610
>1, 9/468, d1=0.287, d2=0.848 g=0.594
>1, 10/468, d1=0.266, d2=0.863 g=0.597
>1, 11/468, d1=0.256, d2=0.853 g=0.634
>1, 12/468, d1=0.284, d2=0.779 g=0.725
>1, 13/468, d1=0.288, d2=0.657 g=0.872
>1, 14/468, d1=0.297, d2=0.535 g=1.042
>1, 15/468, d1=0.312, d2=0.443 g=1.147
>1, 16/468, d1=0.293, d2=0.416 g=1.159
>1, 17/468, d1=0.243, d2=0.440 g=1.107
>1, 18/468, d1=0.370, d2=0.498 g=0.972
>1, 19/468, d1=0.255, d2=0.563 g=0.870
>1, 20/468, d1=0.291, d2=0.622 g=0.788
>1, 21/468, d1=0.291, d2=0.684 g=0.721
>1, 22/468, d1=0.267, d2=0.751 g=0.663
>1, 23/468, d1=0.225, d2=0.853 g=0.591
>1, 24/468, d1=0.167, d2=0.972 g=0.525
>1, 25/468, d1=0.123, d2=1.073 g=0.494
>1, 26/468, d1=0.095, d2=1.068 g=0

>1, 209/468, d1=0.574, d2=0.638 g=0.879
>1, 210/468, d1=0.608, d2=0.694 g=0.850
>1, 211/468, d1=0.597, d2=0.655 g=0.861
>1, 212/468, d1=0.616, d2=0.662 g=0.867
>1, 213/468, d1=0.637, d2=0.670 g=0.889
>1, 214/468, d1=0.675, d2=0.609 g=0.951
>1, 215/468, d1=0.733, d2=0.560 g=1.002
>1, 216/468, d1=0.726, d2=0.537 g=1.048
>1, 217/468, d1=0.719, d2=0.471 g=1.108
>1, 218/468, d1=0.748, d2=0.461 g=1.125
>1, 219/468, d1=0.731, d2=0.464 g=1.104
>1, 220/468, d1=0.749, d2=0.457 g=1.126
>1, 221/468, d1=0.712, d2=0.495 g=1.071
>1, 222/468, d1=0.712, d2=0.487 g=1.047
>1, 223/468, d1=0.682, d2=0.520 g=0.980
>1, 224/468, d1=0.650, d2=0.573 g=0.962
>1, 225/468, d1=0.710, d2=0.565 g=0.928
>1, 226/468, d1=0.628, d2=0.626 g=0.864
>1, 227/468, d1=0.658, d2=0.650 g=0.817
>1, 228/468, d1=0.616, d2=0.730 g=0.745
>1, 229/468, d1=0.687, d2=0.774 g=0.696
>1, 230/468, d1=0.600, d2=0.786 g=0.665
>1, 231/468, d1=0.645, d2=0.870 g=0.647
>1, 232/468, d1=0.664, d2=0.838 g=0.645
>1, 233/468, d1=0.700, d2=0.858 g=0.664


>1, 414/468, d1=0.661, d2=0.596 g=0.817
>1, 415/468, d1=0.646, d2=0.617 g=0.823
>1, 416/468, d1=0.637, d2=0.649 g=0.806
>1, 417/468, d1=0.678, d2=0.635 g=0.778
>1, 418/468, d1=0.638, d2=0.662 g=0.808
>1, 419/468, d1=0.658, d2=0.711 g=0.762
>1, 420/468, d1=0.636, d2=0.675 g=0.756
>1, 421/468, d1=0.630, d2=0.675 g=0.756
>1, 422/468, d1=0.688, d2=0.695 g=0.737
>1, 423/468, d1=0.662, d2=0.685 g=0.757
>1, 424/468, d1=0.638, d2=0.723 g=0.745
>1, 425/468, d1=0.682, d2=0.681 g=0.736
>1, 426/468, d1=0.656, d2=0.695 g=0.758
>1, 427/468, d1=0.684, d2=0.693 g=0.728
>1, 428/468, d1=0.694, d2=0.655 g=0.771
>1, 429/468, d1=0.691, d2=0.679 g=0.760
>1, 430/468, d1=0.680, d2=0.668 g=0.784
>1, 431/468, d1=0.662, d2=0.666 g=0.775
>1, 432/468, d1=0.698, d2=0.645 g=0.777
>1, 433/468, d1=0.664, d2=0.659 g=0.770
>1, 434/468, d1=0.661, d2=0.654 g=0.757
>1, 435/468, d1=0.691, d2=0.646 g=0.781
>1, 436/468, d1=0.701, d2=0.668 g=0.769
>1, 437/468, d1=0.675, d2=0.642 g=0.781
>1, 438/468, d1=0.676, d2=0.638 g=0.770


>2, 154/468, d1=0.653, d2=0.645 g=0.796
>2, 155/468, d1=0.648, d2=0.653 g=0.788
>2, 156/468, d1=0.645, d2=0.664 g=0.776
>2, 157/468, d1=0.618, d2=0.652 g=0.788
>2, 158/468, d1=0.618, d2=0.660 g=0.758
>2, 159/468, d1=0.632, d2=0.638 g=0.755
>2, 160/468, d1=0.657, d2=0.648 g=0.747
>2, 161/468, d1=0.677, d2=0.650 g=0.745
>2, 162/468, d1=0.653, d2=0.693 g=0.734
>2, 163/468, d1=0.624, d2=0.699 g=0.743
>2, 164/468, d1=0.629, d2=0.689 g=0.738
>2, 165/468, d1=0.672, d2=0.698 g=0.718
>2, 166/468, d1=0.657, d2=0.753 g=0.725
>2, 167/468, d1=0.656, d2=0.738 g=0.701
>2, 168/468, d1=0.671, d2=0.701 g=0.711
>2, 169/468, d1=0.694, d2=0.714 g=0.724
>2, 170/468, d1=0.653, d2=0.702 g=0.718
>2, 171/468, d1=0.711, d2=0.698 g=0.715
>2, 172/468, d1=0.693, d2=0.670 g=0.746
>2, 173/468, d1=0.672, d2=0.669 g=0.762
>2, 174/468, d1=0.682, d2=0.667 g=0.766
>2, 175/468, d1=0.667, d2=0.652 g=0.764
>2, 176/468, d1=0.687, d2=0.691 g=0.752
>2, 177/468, d1=0.673, d2=0.676 g=0.751
>2, 178/468, d1=0.685, d2=0.684 g=0.757


>2, 359/468, d1=0.508, d2=0.587 g=0.880
>2, 360/468, d1=0.500, d2=0.578 g=0.896
>2, 361/468, d1=0.539, d2=0.609 g=0.917
>2, 362/468, d1=0.496, d2=0.566 g=0.933
>2, 363/468, d1=0.486, d2=0.589 g=0.862
>2, 364/468, d1=0.513, d2=0.581 g=0.889
>2, 365/468, d1=0.504, d2=0.609 g=0.912
>2, 366/468, d1=0.528, d2=0.584 g=0.933
>2, 367/468, d1=0.538, d2=0.625 g=0.954
>2, 368/468, d1=0.522, d2=0.538 g=0.934
>2, 369/468, d1=0.488, d2=0.590 g=0.915
>2, 370/468, d1=0.461, d2=0.526 g=0.920
>2, 371/468, d1=0.497, d2=0.611 g=0.929
>2, 372/468, d1=0.492, d2=0.583 g=0.956
>2, 373/468, d1=0.482, d2=0.549 g=0.865
>2, 374/468, d1=0.508, d2=0.585 g=0.889
>2, 375/468, d1=0.461, d2=0.626 g=0.876
>2, 376/468, d1=0.516, d2=0.607 g=0.929
>2, 377/468, d1=0.526, d2=0.563 g=0.901
>2, 378/468, d1=0.471, d2=0.602 g=0.889
>2, 379/468, d1=0.537, d2=0.627 g=0.873
>2, 380/468, d1=0.555, d2=0.632 g=0.883
>2, 381/468, d1=0.542, d2=0.665 g=0.950
>2, 382/468, d1=0.520, d2=0.610 g=0.905
>2, 383/468, d1=0.494, d2=0.635 g=0.890


...

>4, 43/468, d1=0.620, d2=0.663 g=0.786
>4, 44/468, d1=0.673, d2=0.635 g=0.761
>4, 45/468, d1=0.639, d2=0.641 g=0.761
>4, 46/468, d1=0.677, d2=0.665 g=0.773
>4, 47/468, d1=0.637, d2=0.665 g=0.765
>4, 48/468, d1=0.655, d2=0.650 g=0.791
>4, 49/468, d1=0.670, d2=0.658 g=0.799
>4, 50/468, d1=0.663, d2=0.668 g=0.817
>4, 51/468, d1=0.679, d2=0.623 g=0.828
>4, 52/468, d1=0.675, d2=0.622 g=0.823
>4, 53/468, d1=0.709, d2=0.628 g=0.794
>4, 54/468, d1=0.674, d2=0.641 g=0.792
>4, 55/468, d1=0.704, d2=0.623 g=0.774
>4, 56/468, d1=0.679, d2=0.649 g=0.774
>4, 57/468, d1=0.659, d2=0.650 g=0.780
>4, 58/468, d1=0.661, d2=0.691 g=0.766
>4, 59/468, d1=0.635, d2=0.676 g=0.782
>4, 60/468, d1=0.671, d2=0.657 g=0.770
>4, 61/468, d1=0.650, d2=0.658 g=0.765
>4, 62/468, d1=0.661, d2=0.667 g=0.763
>4, 63/468, d1=0.633, d2=0.689 g=0.781
>4, 64/468, d1=0.661, d2=0.667 g=0.734
>4, 65/468, d1=0.649, d2=0.673 g=0.752
>4, 66/468, d1=0.652, d2=0.680 g=0.742
>4, 67/468, d1=0.664, d2=0.670 g=0.736
>4, 68/468, d1=0.625, d2=

>4, 250/468, d1=0.652, d2=0.667 g=0.761
>4, 251/468, d1=0.666, d2=0.646 g=0.766
>4, 252/468, d1=0.646, d2=0.688 g=0.762
>4, 253/468, d1=0.658, d2=0.643 g=0.772
>4, 254/468, d1=0.676, d2=0.645 g=0.778
>4, 255/468, d1=0.655, d2=0.658 g=0.798
>4, 256/468, d1=0.671, d2=0.660 g=0.789
>4, 257/468, d1=0.699, d2=0.647 g=0.783
>4, 258/468, d1=0.667, d2=0.671 g=0.781
>4, 259/468, d1=0.693, d2=0.658 g=0.758
>4, 260/468, d1=0.670, d2=0.663 g=0.745
>4, 261/468, d1=0.658, d2=0.686 g=0.747
>4, 262/468, d1=0.673, d2=0.676 g=0.759
>4, 263/468, d1=0.655, d2=0.659 g=0.763
>4, 264/468, d1=0.672, d2=0.671 g=0.741
>4, 265/468, d1=0.623, d2=0.681 g=0.766
>4, 266/468, d1=0.662, d2=0.680 g=0.758
>4, 267/468, d1=0.644, d2=0.679 g=0.750
>4, 268/468, d1=0.632, d2=0.693 g=0.730
>4, 269/468, d1=0.634, d2=0.693 g=0.729
>4, 270/468, d1=0.634, d2=0.722 g=0.713
>4, 271/468, d1=0.665, d2=0.701 g=0.728
>4, 272/468, d1=0.654, d2=0.703 g=0.761
>4, 273/468, d1=0.678, d2=0.678 g=0.811
>4, 274/468, d1=0.665, d2=0.630 g=0.836


>4, 455/468, d1=0.706, d2=0.625 g=0.854
>4, 456/468, d1=0.722, d2=0.637 g=0.811
>4, 457/468, d1=0.664, d2=0.640 g=0.799
>4, 458/468, d1=0.662, d2=0.681 g=0.746
>4, 459/468, d1=0.664, d2=0.691 g=0.738
>4, 460/468, d1=0.691, d2=0.683 g=0.744
>4, 461/468, d1=0.689, d2=0.690 g=0.769
>4, 462/468, d1=0.666, d2=0.649 g=0.786
>4, 463/468, d1=0.668, d2=0.631 g=0.783
>4, 464/468, d1=0.689, d2=0.636 g=0.798
>4, 465/468, d1=0.656, d2=0.636 g=0.790
>4, 466/468, d1=0.671, d2=0.684 g=0.758
>4, 467/468, d1=0.677, d2=0.707 g=0.752
>4, 468/468, d1=0.657, d2=0.711 g=0.704
>5, 1/468, d1=0.650, d2=0.770 g=0.727
>5, 2/468, d1=0.669, d2=0.704 g=0.729
>5, 3/468, d1=0.694, d2=0.680 g=0.779
>5, 4/468, d1=0.685, d2=0.627 g=0.821
>5, 5/468, d1=0.676, d2=0.610 g=0.872
>5, 6/468, d1=0.737, d2=0.597 g=0.863
>5, 7/468, d1=0.713, d2=0.594 g=0.847
>5, 8/468, d1=0.687, d2=0.680 g=0.804
>5, 9/468, d1=0.681, d2=0.670 g=0.770
>5, 10/468, d1=0.675, d2=0.723 g=0.747
>5, 11/468, d1=0.691, d2=0.701 g=0.739
>5, 12/468, d1=0.677

>5, 195/468, d1=0.691, d2=0.676 g=0.725
>5, 196/468, d1=0.686, d2=0.718 g=0.730
>5, 197/468, d1=0.676, d2=0.651 g=0.754
>5, 198/468, d1=0.640, d2=0.670 g=0.787
>5, 199/468, d1=0.651, d2=0.647 g=0.810
>5, 200/468, d1=0.653, d2=0.647 g=0.812
>5, 201/468, d1=0.643, d2=0.628 g=0.785
>5, 202/468, d1=0.666, d2=0.660 g=0.790
>5, 203/468, d1=0.672, d2=0.698 g=0.747
>5, 204/468, d1=0.656, d2=0.699 g=0.760
>5, 205/468, d1=0.695, d2=0.717 g=0.723
>5, 206/468, d1=0.639, d2=0.680 g=0.759
>5, 207/468, d1=0.693, d2=0.650 g=0.788
>5, 208/468, d1=0.703, d2=0.623 g=0.840
>5, 209/468, d1=0.661, d2=0.581 g=0.854
>5, 210/468, d1=0.671, d2=0.616 g=0.881
>5, 211/468, d1=0.683, d2=0.581 g=0.845
>5, 212/468, d1=0.678, d2=0.598 g=0.832
>5, 213/468, d1=0.698, d2=0.639 g=0.789
>5, 214/468, d1=0.694, d2=0.651 g=0.770
>5, 215/468, d1=0.685, d2=0.697 g=0.766
>5, 216/468, d1=0.687, d2=0.684 g=0.755
>5, 217/468, d1=0.679, d2=0.659 g=0.767
>5, 218/468, d1=0.676, d2=0.644 g=0.778
>5, 219/468, d1=0.645, d2=0.642 g=0.766


>5, 400/468, d1=0.660, d2=0.637 g=0.803
>5, 401/468, d1=0.662, d2=0.646 g=0.795
>5, 402/468, d1=0.702, d2=0.646 g=0.784
>5, 403/468, d1=0.700, d2=0.666 g=0.777
>5, 404/468, d1=0.667, d2=0.672 g=0.775
>5, 405/468, d1=0.677, d2=0.677 g=0.767
>5, 406/468, d1=0.686, d2=0.654 g=0.792
>5, 407/468, d1=0.690, d2=0.668 g=0.765
>5, 408/468, d1=0.688, d2=0.673 g=0.760
>5, 409/468, d1=0.667, d2=0.665 g=0.749
>5, 410/468, d1=0.658, d2=0.671 g=0.764
>5, 411/468, d1=0.669, d2=0.683 g=0.747
>5, 412/468, d1=0.636, d2=0.666 g=0.758
>5, 413/468, d1=0.684, d2=0.665 g=0.767
>5, 414/468, d1=0.640, d2=0.662 g=0.772
>5, 415/468, d1=0.649, d2=0.655 g=0.764
>5, 416/468, d1=0.661, d2=0.657 g=0.761
>5, 417/468, d1=0.676, d2=0.666 g=0.775
>5, 418/468, d1=0.673, d2=0.652 g=0.776
>5, 419/468, d1=0.662, d2=0.646 g=0.792
>5, 420/468, d1=0.678, d2=0.651 g=0.769
>5, 421/468, d1=0.660, d2=0.655 g=0.789
>5, 422/468, d1=0.663, d2=0.658 g=0.771
>5, 423/468, d1=0.675, d2=0.637 g=0.797
>5, 424/468, d1=0.679, d2=0.649 g=0.769


In [253]:
# example of loading the generator model and generating images

from numpy.random import randn
from numpy.random import randint
from keras.models import load_model
from matplotlib import pyplot

# generate points in latent space as input for the generator
def generate_latent_points2(latent_dim, n_samples, n_classes=10):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    z_input = x_input.reshape(n_samples, latent_dim)
    # generate labels
    labels = randint(0, n_classes, n_samples)
    return [z_input, np.vstack((labels, labels)).T]

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=10):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    z_input = x_input.reshape(n_samples, latent_dim)
    # generate labels
    labels = randint(0, n_classes, n_samples)
    return [z_input, labels]

# create and save a plot of generated images
def save_plot(examples, n):
    # plot images
    for i in range(n * n):
        # define subplot
        pyplot.subplot(n, n, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
    pyplot.show()


In [275]:
# load model
model = load_model('cgan_models/cgan_generator.h5')

In [280]:
latent_points, labels = generate_latent_points(100, 2)
# specify labels
labels = np.array([1, 1]).reshape()
# generate images
X = model.predict([latent_points, labels])
# scale from [-1,1] to [0,1]
X = (X + 1) / 2.0
# plot the result
save_plot(X, 4)


ValueError: Error when checking input: expected input_105 to have shape (2,) but got array with shape (1,)

In [279]:
model.summary()

Model: "model_73"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_106 (InputLayer)          (None, 100)          0                                            
__________________________________________________________________________________________________
input_105 (InputLayer)          (None, 2)            0                                            
__________________________________________________________________________________________________
dense_106 (Dense)               (None, 6272)         633472      input_106[0][0]                  
__________________________________________________________________________________________________
embedding_55 (Embedding)        (None, 2, 50)        500         input_105[0][0]                  
___________________________________________________________________________________________

In [281]:
labels.shape

(2,)